# 예제 7-5 SVM 모형

### 기본 라이브러리 불러오기 

In [1]:
import pandas as pd
import seaborn as sns

## Step 1 데이터 준비/기본 설정 

### load_dataset 함수를 이용하여 데이터프레임 전환 

In [2]:
df = sns.load_dataset('titanic')

## Step 2 데이터 탐색/전처리 

### NaN값이 많은 deck열 삭제, embarked와 내용이 겹치는 embark_town 열 삭제 

In [3]:
rdf = df.drop(['deck', 'embark_town'], axis=1)

### age 열에 나이 데이터가 없는 모든 행 삭제  

In [4]:
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

### embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기 

In [5]:
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)

## Step 3 분석에 사용할 속성 선택 

### 분석에 활용할 열 선택 

In [6]:
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

### 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환 

In [7]:
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

In [9]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

In [10]:
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


## 데이터셋 구분 - 훈련용/검증용

### 속성 선택 

In [11]:
X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male',
       'town_C', 'town_Q', 'town_S']]
y = ndf['survived']

### 설명 변수 데이터를 정규화 

In [12]:
from sklearn import preprocessing

In [13]:
X = preprocessing.StandardScaler().fit(X).transform(X)

### train data와 test data로 구분 

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                   random_state=10)

In [15]:
print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


## SVM 분류 모형 - sklearn 사용 

### sklearn 라이브러리 에서 SVM 분류 모형 가져오기 

In [16]:
from sklearn import svm

### 모형 객체 생성(kernel='rbf' 적용) 

In [17]:
svm_model = svm.SVC(kernel='rbf')

데이터를 벡터 공간으로 매핑하는 함수를 커널이라고 하는데 , RBF(Radial Basis Function)함수를 적용한다. 

### train data를 가지고 모형 학습 

In [20]:
svm_model.fit(X_train, y_train)

SVC()

### test data를 가지고 y_hat 예측(분류) 

In [21]:
y_hat = svm_model.predict(X_test)

In [22]:
print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


10개 중 8개가 일치한다.

### 모형 성능 평가 - Confusion Matrix 계산 

In [23]:
from sklearn import metrics

In [24]:
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(svm_matrix)

[[120   5]
 [ 35  55]]


### 모형 성능 평가 - 평가 지표 계산 

In [26]:
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



미생존자 예측의 정확도가 0.86, 생존자 예측의 정확도가 0.73이다.
KNN모형의 예측 능력과 큰 차이는 없다.